In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
!hdfs dfs -ls /tmp
!hdfs dfs -mkdir /tmp/wordcount
!hdfs dfs -ls /tmp/

Found 4 items
drwxrwx---   - root supergroup          0 2020-02-18 19:12 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-18 19:19 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-02-18 21:16 /tmp/output
drwxr-xr-x   - root supergroup          0 2020-02-18 19:38 /tmp/wordcount
mkdir: `/tmp/wordcount': File exists
Found 4 items
drwxrwx---   - root supergroup          0 2020-02-18 19:12 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-18 19:19 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-02-18 21:16 /tmp/output
drwxr-xr-x   - root supergroup          0 2020-02-18 19:38 /tmp/wordcount


In [3]:
!hdfs dfs -copyFromLocal *.csv  /tmp/wordcount/

copyFromLocal: `/tmp/wordcount/tbl0.csv': File exists
copyFromLocal: `/tmp/wordcount/tbl1.csv': File exists


In [4]:
%%hive

DROP TABLE IF EXISTS tbl0;
DROP TABLE IF EXISTS tbl1;

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 13.977 seconds
DROP TABLE IF EXISTS tbl1;
OK
Time taken: 0.177 seconds


In [5]:
%%hive

CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;

CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 0.92 seconds
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
Loading data to table default.tbl0
OK
Time taken: 1.666 seconds


In [6]:
%%hive

CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;

CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 0.089 seconds
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;
Loading data to table default.tbl1
OK
Time taken: 1.086 seconds


In [21]:
%%hive
DROP TABLE IF EXISTS lets_map_vals;
CREATE TABLE lets_map_vals AS
SELECT a.c2, SUM(a.v)
FROM
(SELECT c2, v
FROM tbl0
LATERAL VIEW explode(c6) tbl0 AS k, v) a
GROUP BY a.c2;

DROP TABLE IF EXISTS lets_map_vals;
OK
Time taken: 0.019 seconds
CREATE TABLE lets_map_vals AS
SELECT a.c2, SUM(a.v)
FROM
(SELECT c2, v
FROM tbl0
LATERAL VIEW explode(c6) tbl0 AS k, v) a
GROUP BY a.c2;
Query ID = root_20200218213237_f0f998d0-83d1-4121-b28e-3b4445e4256d
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1582053079980_0025, Tracking URL = http://fbebe19f3edd:8088/proxy/application_1582053079980_0025/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1582053079980_0025
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-02-18 21:32:46,055 Stage-1 map = 0%,  reduce = 0%
20

In [22]:
%%hive
INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM lets_map_vals;

INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM lets_map_vals;
Query ID = root_20200218213313_bb12d4b2-e327-47ce-862c-1eb6f5c45605
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1582053079980_0026, Tracking URL = http://fbebe19f3edd:8088/proxy/application_1582053079980_0026/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1582053079980_0026
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-02-18 21:33:22,781 Stage-1 map = 0%,  reduce = 0%
2020-02-18 21:33:30,550 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 2.07 sec
MapReduce Total cumulative CPU time: 2 seconds 70 msec
Ended Job = job_1582053079980_0026
Stage-3 is selected by condition resolver.
Stage-2 is filtered out by condition resolver.
Stage-4 is filtered out by condition resolver.
Moving data to directory hdfs://0.0.0.0:9000/tmp/output/.hive-staging

In [23]:
!hive -S -e 'source output.hql'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
File: output.hql is not a file.


In [24]:
!hadoop fs -copyToLocal /tmp/output output
!ls output/*

output/000000_0


In [25]:
!cat output/000000_0

A,20
B,200
C,130
D,220
